This notebook will create the subset of images used in train and test, from NSD saved as pickle files.

In [17]:
import csv
from scipy.io import loadmat
import numpy as np
import os

In [21]:
DSET_PATH = "/srv/eeg_reconstruction/shared/biosemi-dataset/eeg2"

In [13]:
def load_csv_to_list(csv_filepath):
    """
    Reads a CSV file and converts it into a list of tuples.

    Parameters:
    - csv_filepath: Path to the CSV file.

    Returns:
    - A list of tuples, where each tuple contains [image number, cocoId].
    """
    data = []
    with open(csv_filepath, mode='r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader, None)
        for idx, row in enumerate(reader):
            # Convert the first, second columns to int 
            if row[0] == '':
                print(f"Row index: {idx} is blank")
                continue
            else:
                data.append((int(row[0]), int(row[1])))
    return data


def get_coco_id(csv_data, index):
    """
    Returns the COCO ID for a given index from CSV data.

    Parameters:
    - csv_data: A list of tuples/lists where each tuple/list contains [image number, cocoId].
    - index: The 0-based image number to find the COCO ID for.

    Returns:
    - The COCO ID corresponding to the given index.
    """
    for row in csv_data:
        if row[0] == index:
            return row[1]
    return None  # Return None if the index is not found

# Load conversion CSV data and .mat contents outside the loop to avoid reloading for each file
conversion_csv_filepath = '../final_dataset/nsd_coco_conversion.csv'  
conversion_csv_data = load_csv_to_list(conversion_csv_filepath)

nsd_mat_file_path = '../final_dataset/nsd_expdesign.mat'
nsd_mat_contents = loadmat(nsd_mat_file_path)
nsd_indices = nsd_mat_contents["sharedix"][0] # We look at the shared 1000 images since we only looked at the shared1000
nsd_indices = nsd_indices - 1 # NSD Indices count from 1

In [18]:
coco_indices = np.array([get_coco_id(conversion_csv_data, nsd_id) for nsd_id in nsd_indices])

In [22]:
np.save(os.path.join(DSET_PATH, "shared1000_indices.npy"), coco_indices)